#Pre Processing

In [ ]:
!pip install twint
!pip install sastrawi
!pip install keras
!pip freeze
!pip install nest_asyncio
!pip install NDETCStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 263 kB 9.4 MB/s 
     |████████████████████████████████| 384 kB 61.6 MB/s 
     |████████████████████████████████| 288 kB 57.2 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 59 kB 5.7 MB/s 
     |████████████████████████████████| 140 kB 19.1 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33929 sha256=6a0bb186436b40cbc97330e8b6412bf76ea93252114554fb6edc8ebfd6b2421b
  Stored in directory: /root/.cache/pip/wheels/44/fc/77/99887a36b5c265a87516158858697d1a0b8f32c4d4dbddbb24
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=6d665f95f524d2d37018dbf3c94fb58287831c8c4aa4b236af79759ed544076c
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9

In [ ]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style = 'whitegrid')
import twint
import nest_asyncio
nest_asyncio.apply()
import ast

from NDETCStemmer import NDETCStemmer
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud
from  tensorflow.keras.optimizers import Adam, RMSprop, SGD

from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/...')

In [ ]:
df = df[['reviewId','content', 'label']]
df = df[1:50000]

In [ ]:
#Konversi label

df1 = df.copy()
arr = [df1]

for dataset in arr:    
  dataset.loc[(dataset['label'] == 'negatif'), 'tag'] = 0
  dataset.loc[(dataset['label'] == 'netral'), 'tag'] = 1
  dataset.loc[(dataset['label'] == 'positif'), 'tag'] = 2

df1['tag'] = df1['tag'].astype(int)
df = df1

In [ ]:
df.head()

,reviewId,content,label,tag
1,gp:AOqpTOHpOwsdcBZG4NLalocx4TgZL8Pz2ToyOiX8jKp...,Okkkk,positif,2
2,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2
3,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0
4,gp:AOqpTOEuUCXRkYNNUg2CRm1qz7P-KyYQPZY-RqfvZxG...,Keren,positif,2
5,gp:AOqpTOEnAm6bRqZjqDYfLZyVtqhX3bY-cb-UIFgbRMp...,😋😋😋😋😋😋😋😆😗😗😗😗😙😙😙,netral,1


In [ ]:
df.dtypes

reviewId    object
content     object
label       object
tag          int64
dtype: object

In [ ]:
dup = df.duplicated(subset=['reviewId'])
print(dup.sum())

0


In [ ]:
konten = df[df['content'].str.contains('konten')]
video = df[df['content'].str.contains('video')]
konten

,reviewId,content,label,tag
27,gp:AOqpTOFBRe2yZehNNCrUFOHy22fsXrCToqWah2Emuco...,Aku bisa membuat hariku menjadi lebih senang k...,positif,2
79,gp:AOqpTOGo-Djrt4K4ah5vUI4J59FwBKHvSKO_o51PIbv...,Saya suka banget apk ini dan saya menjadi keta...,positif,2
216,gp:AOqpTOEJ7Ov72IiI9K1eRR0VRp1tvmiftP5gXpsLKFL...,Gua pengen video gua Fyp terus,negatif,0
358,gp:AOqpTOE0N-lVMevFXPbg2JG19sV4DgAQwFjHQv8M1-Y...,"Nguras kouta, baru scroll 3-5video udah 200mb ...",negatif,0
390,gp:AOqpTOHPWR2OwiC5VoHzqbUcYRsGD4mmbvF4h--mays...,"Bagus, kalo bisa mendapatkan cuan dari setiap ...",positif,2
...,...,...,...,...
49619,gp:AOqpTOGWneWrBSSYb3MaLmKvugHf8OS6FG34BmC3bov...,"saya ingin komplem masalah FYP yah,ini entah k...",negatif,0
49657,gp:AOqpTOH3t9WOTtK3HFno6bClzp4WgYuI28zYyWrTZxJ...,Sistem simpannaya /tambahke favorit aku kurang...,negatif,0
49819,gp:AOqpTOGXMZHi49osD0_Yj_OuL7u4mWYq1bNCv3BEYeK...,Ketikan orang buat Vidio random banya yang lih...,negatif,0
49859,gp:AOqpTOHCXsRRzM0hXGrGplUiNVtsxUEHt2pdKzHDYXZ...,Semoga video yang saya upload di platform TikT...,positif,2


In [ ]:
konten.groupby(['label'])['label'].count()

label
negatif    457
netral     105
positif    401
Name: label, dtype: int64

In [ ]:
import pandas as pd
slangwords1 = pd.read_csv('/content/drive/MyDrive/...') 
slangwords1 = slangwords1.set_index('kataAlay')
slangwords2 = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
slangwords2 = slangwords2.filter(['slang', 'formal'], axis=1)
slangwords2 = slangwords2.drop_duplicates(subset=['slang'], keep='first')
slangwords2 = slangwords2.set_index('slang')
stopword1   = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
custom_word = ['nya', 'tolong', 'iya', 'guna', 'kasih', 'buka', 'hari', 'sih', 'mohon', 'baru', 'tiktok', 'aplikasi', 'apl', 'apk', 'tik', 'tok', 'konten', 'video']

In [ ]:
slangwords1

,kataBaik
kataAlay,
pjj,pembelajaran jarak jauh
jan,jangan
emg,memang
y,ya
lu,kamu
...,...
dsn,dsn
kec,kecamatan
blk,blok


In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def cleaningText(text):
    text = remove_emoji(text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = re.sub(r'(\w)(\1{2,})', r"\1", text) #will replace repated char
    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def filteringText(text): # Remove stopwors in a text
    #stopword
    text_split = text.split(' ')
    for i in range(len(text_split)):
      if text_split[i] in slangwords1.index:
        text_split[i] = slangwords1.loc[text_split[i]]['kataBaik']
      elif text_split[i] in slangwords2.index:
        text_split[i] = slangwords2.loc[text_split[i]]['formal']
      else:
        pass
    temp_text_split = []
    for i in range(len(text_split)):
      if (text_split[i] not in stopword1) and (text_split[i] not in custom_word) and (type(text_split[i]) == str):
        temp_text_split.append(text_split[i])
    text = ' '.join(temp_text_split)
    return text

#PreProcessing
def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    # stemmer = NDETCStemmer()
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
  # sentence = ast.literal_eval(list_words)
  sentence = ' '.join(list_words)
  return sentence

In [ ]:
df['text_clean'] = df['content'].apply(cleaningText)
df['text_clean'] = df['text_clean'].apply(casefoldingText)
# df['text_clean'] = df['text_clean'].apply(convertToSlangword)
df['text_clean'] = df['text_clean'].apply(filteringText)
# df.drop(['content'], axis = 1, inplace = True)

df['text_preprocessed'] = df['text_clean'].apply(tokenizingText)
df['text_preprocessed'] = df['text_preprocessed'].apply(stemmingText)
df['text_used'] = df['text_preprocessed'].apply(toSentence)

# drop duplicates/spams tweets
df.drop_duplicates(subset = 'text_clean', inplace = True)

In [ ]:
df

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
1,gp:AOqpTOHpOwsdcBZG4NLalocx4TgZL8Pz2ToyOiX8jKp...,Okkkk,positif,2,ok,[ok],ok
2,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2,bagus banget pokoknya,"[bagus, banget, pokok]",bagus banget pokok
3,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0,permainan babi,"[main, babi]",main babi
4,gp:AOqpTOEuUCXRkYNNUg2CRm1qz7P-KyYQPZY-RqfvZxG...,Keren,positif,2,keren,[keren],keren
5,gp:AOqpTOEnAm6bRqZjqDYfLZyVtqhX3bY-cb-UIFgbRMp...,😋😋😋😋😋😋😋😆😗😗😗😗😙😙😙,netral,1,,[],
...,...,...,...,...,...,...,...
49992,gp:AOqpTOHwMXaJSQ9tHpbwpaHzcq0aRzxci6xqh2DZ1n2...,Seru2an dan bertambah teman virtual,positif,2,seruan bertambah teman virtual,"[seru, tambah, teman, virtual]",seru tambah teman virtual
49993,gp:AOqpTOE3XpPZW4YJwJ5fqTYrvtX9Z-IAwuqi2tbw89D...,Ngentottt siateh,negatif,0,ngentot siateh,"[ngentot, siateh]",ngentot siateh
49994,gp:AOqpTOEM69K-OsZdiaftaT_BTpt6_snJfPSJmtRMiXU...,Aplikasinya bagus sekali!!!!!!!!!!!!!!!!!!!!!!...,positif,2,aplikasinya bagus sekali¡±’”¢° slezrd€€÷√√€π÷€...,"[aplikasi, bagus, sekali, , , , slezrd, kg ken...",aplikasi bagus sekali slezrd kg kenapa liha...
49995,gp:AOqpTOFa6V8C93IFNEyXY3iiFufXvvK_E-QnYLJbsQQ...,Gahbisa masuk akun aku,negatif,0,gahbisa masuk akun,"[gahbisa, masuk, akun]",gahbisa masuk akun


In [ ]:
df = df.drop(26028)
df = df.reset_index(drop=True)

In [ ]:
df

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
0,gp:AOqpTOHpOwsdcBZG4NLalocx4TgZL8Pz2ToyOiX8jKp...,Okkkk,positif,2,ok,[ok],ok
1,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2,bagus banget pokoknya,"[bagus, banget, pokok]",bagus banget pokok
2,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0,permainan babi,"[main, babi]",main babi
3,gp:AOqpTOEuUCXRkYNNUg2CRm1qz7P-KyYQPZY-RqfvZxG...,Keren,positif,2,keren,[keren],keren
4,gp:AOqpTOEnAm6bRqZjqDYfLZyVtqhX3bY-cb-UIFgbRMp...,😋😋😋😋😋😋😋😆😗😗😗😗😙😙😙,netral,1,,[],
...,...,...,...,...,...,...,...
26025,gp:AOqpTOHUr2AH7s0ANXk4YgiXAHVf4krAStLXXxbjcqr...,Apk sesat mending download Coco fun :v,negatif,0,sesat lebih baik download coco fun v,"[sesat, lebih, baik, download, coco, fun, v]",sesat lebih baik download coco fun v
26026,gp:AOqpTOHwMXaJSQ9tHpbwpaHzcq0aRzxci6xqh2DZ1n2...,Seru2an dan bertambah teman virtual,positif,2,seruan bertambah teman virtual,"[seru, tambah, teman, virtual]",seru tambah teman virtual
26027,gp:AOqpTOE3XpPZW4YJwJ5fqTYrvtX9Z-IAwuqi2tbw89D...,Ngentottt siateh,negatif,0,ngentot siateh,"[ngentot, siateh]",ngentot siateh
26028,gp:AOqpTOFa6V8C93IFNEyXY3iiFufXvvK_E-QnYLJbsQQ...,Gahbisa masuk akun aku,negatif,0,gahbisa masuk akun,"[gahbisa, masuk, akun]",gahbisa masuk akun


In [ ]:
teks = df.loc[4,'text_used']
teks

''

In [ ]:
# Untuk perbandingan (Drop column list kata = 0)
df1 = df.copy()
df1["WordsCount"]=df["text_preprocessed"].apply(lambda x: len(x))
df1 = df1[df1["WordsCount"] > 1]
df1 = df1[df1.WordsCount != 0]
df1 = df1.drop(['WordsCount'], axis=1)

In [ ]:
df1 = df1.reset_index(drop=True)
df = df1
df

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
0,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2,bagus banget pokoknya,"[bagus, banget, pokok]",bagus banget pokok
1,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0,permainan babi,"[main, babi]",main babi
2,gp:AOqpTOEsUd_G8nZ3RdkMFdN9CEixth7_9FipWJpCgJ6...,Tiktok. Pohon katus,netral,1,pohon katus,"[pohon, katus]",pohon katus
3,gp:AOqpTOEWHfXgz1qYZfsXVHExYqXohT_XgbORTILu7NJ...,tolong jangan boros kali lah makan paket,negatif,0,boros kali makan paket,"[boros, kali, makan, paket]",boros kali makan paket
4,gp:AOqpTOEfEwd-7QBj12PPlUzopd7piWH4RXx28RQNUAQ...,Tteitfbdryrlfrhtwr 😗😗rlur,netral,1,tteitfbdryrlfrhtwr rlur,"[tteitfbdryrlfrhtwr, rlur]",tteitfbdryrlfrhtwr rlur
...,...,...,...,...,...,...,...
22533,gp:AOqpTOHUr2AH7s0ANXk4YgiXAHVf4krAStLXXxbjcqr...,Apk sesat mending download Coco fun :v,negatif,0,sesat lebih baik download coco fun v,"[sesat, lebih, baik, download, coco, fun, v]",sesat lebih baik download coco fun v
22534,gp:AOqpTOHwMXaJSQ9tHpbwpaHzcq0aRzxci6xqh2DZ1n2...,Seru2an dan bertambah teman virtual,positif,2,seruan bertambah teman virtual,"[seru, tambah, teman, virtual]",seru tambah teman virtual
22535,gp:AOqpTOE3XpPZW4YJwJ5fqTYrvtX9Z-IAwuqi2tbw89D...,Ngentottt siateh,negatif,0,ngentot siateh,"[ngentot, siateh]",ngentot siateh
22536,gp:AOqpTOFa6V8C93IFNEyXY3iiFufXvvK_E-QnYLJbsQQ...,Gahbisa masuk akun aku,negatif,0,gahbisa masuk akun,"[gahbisa, masuk, akun]",gahbisa masuk akun


In [ ]:
df = df.drop(4)
df = df.reset_index(drop=True)
df

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
0,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2,bagus banget pokoknya,"[bagus, banget, pokok]",bagus banget pokok
1,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0,permainan babi,"[main, babi]",main babi
2,gp:AOqpTOEsUd_G8nZ3RdkMFdN9CEixth7_9FipWJpCgJ6...,Tiktok. Pohon katus,netral,1,pohon katus,"[pohon, katus]",pohon katus
3,gp:AOqpTOEWHfXgz1qYZfsXVHExYqXohT_XgbORTILu7NJ...,tolong jangan boros kali lah makan paket,negatif,0,boros kali makan paket,"[boros, kali, makan, paket]",boros kali makan paket
4,gp:AOqpTOHl33Ag2bFocaxz1d0VRR1TJ3wZlO0UwpBp77V...,Ngakak liat pendewa naruto kena infinite gojou...,netral,1,tertawa lihat pendewa naruto kena infinite goj...,"[tertawa, lihat, dewa, naruto, kena, infinite,...",tertawa lihat dewa naruto kena infinite gojou ...
...,...,...,...,...,...,...,...
22532,gp:AOqpTOHUr2AH7s0ANXk4YgiXAHVf4krAStLXXxbjcqr...,Apk sesat mending download Coco fun :v,negatif,0,sesat lebih baik download coco fun v,"[sesat, lebih, baik, download, coco, fun, v]",sesat lebih baik download coco fun v
22533,gp:AOqpTOHwMXaJSQ9tHpbwpaHzcq0aRzxci6xqh2DZ1n2...,Seru2an dan bertambah teman virtual,positif,2,seruan bertambah teman virtual,"[seru, tambah, teman, virtual]",seru tambah teman virtual
22534,gp:AOqpTOE3XpPZW4YJwJ5fqTYrvtX9Z-IAwuqi2tbw89D...,Ngentottt siateh,negatif,0,ngentot siateh,"[ngentot, siateh]",ngentot siateh
22535,gp:AOqpTOFa6V8C93IFNEyXY3iiFufXvvK_E-QnYLJbsQQ...,Gahbisa masuk akun aku,negatif,0,gahbisa masuk akun,"[gahbisa, masuk, akun]",gahbisa masuk akun


In [ ]:
df = df.dropna()
df.isnull().sum()
df = df.reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/Kuliah/Proposal/TA/dataset/CleanData/df_tiktok_clean_used_2.csv', index = False)
df

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
0,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2,bagus banget pokoknya,"[bagus, banget, pokok]",bagus banget pokok
1,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0,permainan babi,"[main, babi]",main babi
2,gp:AOqpTOEsUd_G8nZ3RdkMFdN9CEixth7_9FipWJpCgJ6...,Tiktok. Pohon katus,netral,1,pohon katus,"[pohon, katus]",pohon katus
3,gp:AOqpTOEWHfXgz1qYZfsXVHExYqXohT_XgbORTILu7NJ...,tolong jangan boros kali lah makan paket,negatif,0,boros kali makan paket,"[boros, kali, makan, paket]",boros kali makan paket
4,gp:AOqpTOHl33Ag2bFocaxz1d0VRR1TJ3wZlO0UwpBp77V...,Ngakak liat pendewa naruto kena infinite gojou...,netral,1,tertawa lihat pendewa naruto kena infinite goj...,"[tertawa, lihat, dewa, naruto, kena, infinite,...",tertawa lihat dewa naruto kena infinite gojou ...
...,...,...,...,...,...,...,...
22532,gp:AOqpTOHUr2AH7s0ANXk4YgiXAHVf4krAStLXXxbjcqr...,Apk sesat mending download Coco fun :v,negatif,0,sesat lebih baik download coco fun v,"[sesat, lebih, baik, download, coco, fun, v]",sesat lebih baik download coco fun v
22533,gp:AOqpTOHwMXaJSQ9tHpbwpaHzcq0aRzxci6xqh2DZ1n2...,Seru2an dan bertambah teman virtual,positif,2,seruan bertambah teman virtual,"[seru, tambah, teman, virtual]",seru tambah teman virtual
22534,gp:AOqpTOE3XpPZW4YJwJ5fqTYrvtX9Z-IAwuqi2tbw89D...,Ngentottt siateh,negatif,0,ngentot siateh,"[ngentot, siateh]",ngentot siateh
22535,gp:AOqpTOFa6V8C93IFNEyXY3iiFufXvvK_E-QnYLJbsQQ...,Gahbisa masuk akun aku,negatif,0,gahbisa masuk akun,"[gahbisa, masuk, akun]",gahbisa masuk akun


In [ ]:
print("done")

#Advanced Cleaning

In [ ]:
import pandas as pd
df = pd.read_csv('//content/drive/MyDrive/...') 

In [ ]:
df1 = df.copy()
df1["WordsCount"]=df["text_used"].apply(lambda x: len(x))
df1 = df1.reset_index(drop=True)

#For Check Len N Data
# df1 = df1[df1["WordsCount"] > 0]
# df1 = df1[df1["WordsCount"] <= 250]
# df1 = df1[df1.WordsCount != 0]
# df1 = df1.drop(['WordsCount'], axis=1)
# df1 = df1.reset_index(drop=True)
# df = df1.copy()

In [ ]:
wordcount = df1[df1["WordsCount"]> 250]
cek = wordcount.iloc[:, 1:2]
cek = cek.reset_index(drop=True)

In [ ]:
#data diatas 250 Character
cek.sample(10) 

,content
39,Ajg BBM Kxjjcbfidjdbfjfbfbfifhfbf chrbdieoosow...
0,hsjsjsjidbdndjkdkfjfjudowksdnjdkdklslkskdkdkdk...
94,Sangat senang..karna kalau udah main tiktok ba...
3,Keren🗿 Tapi ada satu hal yg kita benci yaitu 1...
100,game nya gak seru banget pas aku coba ngelek J...
87,Tidnjdjdjdjdhxjhxhxhdhxhdhdhdjjdjxjxhxhcxhxhhf...
72,Bintangnya Saya Kurangin Soalnya pertama launc...
10,Tiktok menurut gw sosmed yang teraneh se-Indon...
86,Lalalalalallalalalalalalalallalalallalalalalal...
108,Kon ga bagus aku ga intal xnxbbc vs vxvdddgddg...


In [ ]:
cek.index[[0,2,8,46,98]]

Int64Index([0, 2, 8, 46, 98], dtype='int64')

In [ ]:
cek.iloc[[0,2,8,46,98]]

,content
0,hsjsjsjidbdndjkdkfjfjudowksdnjdkdklslkskdkdkdk...
2,Hdjdkfkdudkjxkdkekdkdjdmjdjdndndnlskfnjdndopwp...
8,RISKY rsyshdydyhxcyivufuisoddfhjfjgjgjgjgjgkgk...
46,",hahahahahahahahahahahahahahahahahahahahahahah..."
98,Eewweeaqwr Yudi Downer Yudith Freddy Ipul xrou...


In [ ]:
df = df.drop(df.index[[9752]])
df = df.reset_index(drop=True)

In [ ]:
length_of_the_messages = df['text_used'].str.split("\\s+")
print("Max number of words = ", length_of_the_messages.str.len().max())
print("Index = ", length_of_the_messages.str.len().idxmax())

Max number of words =  49
Index =  3965


In [ ]:
df.loc[3965,'label']

'netral'

In [ ]:
df.loc[3965,:]

reviewId             gp:AOqpTOEbavSWZu0JH3ZdErSao58lSBo5wob2RKOmXYb...
content              tHut ini v ya 56 hr vvbb di TV inie ya yg un b...
label                                                           netral
tag                                                                  1
text_clean           thut v  vvbb tv inie ujian nasional bu eh v eh...
text_preprocessed    ['thut', 'v', 'vvbb', 'tv', 'inie', 'uji', 'na...
text_used            thut v vvbb tv inie uji nasional bu eh v eh bu...
Name: 3965, dtype: object

In [ ]:
df = df.dropna()
df.isnull().sum()
df = df.reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/Kuliah/Proposal/TA/dataset/CleanData/df_tiktok_clean_used_3.csv', index = False)
df

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
0,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2,bagus banget pokoknya,"['bagus', 'banget', 'pokok']",bagus banget pokok
1,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0,permainan babi,"['main', 'babi']",main babi
2,gp:AOqpTOEsUd_G8nZ3RdkMFdN9CEixth7_9FipWJpCgJ6...,Tiktok. Pohon katus,netral,1,pohon katus,"['pohon', 'katus']",pohon katus
3,gp:AOqpTOEWHfXgz1qYZfsXVHExYqXohT_XgbORTILu7NJ...,tolong jangan boros kali lah makan paket,negatif,0,boros kali makan paket,"['boros', 'kali', 'makan', 'paket']",boros kali makan paket
4,gp:AOqpTOHl33Ag2bFocaxz1d0VRR1TJ3wZlO0UwpBp77V...,Ngakak liat pendewa naruto kena infinite gojou...,netral,1,tertawa lihat pendewa naruto kena infinite goj...,"['tertawa', 'lihat', 'dewa', 'naruto', 'kena',...",tertawa lihat dewa naruto kena infinite gojou ...
...,...,...,...,...,...,...,...
22419,gp:AOqpTOHUr2AH7s0ANXk4YgiXAHVf4krAStLXXxbjcqr...,Apk sesat mending download Coco fun :v,negatif,0,sesat lebih baik download coco fun v,"['sesat', 'lebih', 'baik', 'download', 'coco',...",sesat lebih baik download coco fun v
22420,gp:AOqpTOHwMXaJSQ9tHpbwpaHzcq0aRzxci6xqh2DZ1n2...,Seru2an dan bertambah teman virtual,positif,2,seruan bertambah teman virtual,"['seru', 'tambah', 'teman', 'virtual']",seru tambah teman virtual
22421,gp:AOqpTOE3XpPZW4YJwJ5fqTYrvtX9Z-IAwuqi2tbw89D...,Ngentottt siateh,negatif,0,ngentot siateh,"['ngentot', 'siateh']",ngentot siateh
22422,gp:AOqpTOFa6V8C93IFNEyXY3iiFufXvvK_E-QnYLJbsQQ...,Gahbisa masuk akun aku,negatif,0,gahbisa masuk akun,"['gahbisa', 'masuk', 'akun']",gahbisa masuk akun


#Split Data

In [ ]:
df = df.dropna()
df.isnull().sum()
# df = df.drop(['No', 'reviewId', 'userName'], axis=1)
df = df.reset_index(drop=True)

In [ ]:
df

,content,label,text_clean,text_preprocessed,tag,text_used
0,Okkkk,positif,ok,['ok'],1,ok
1,Bagus bangettttttttttt pokonya,positif,bagus banget pokoknya,"['bagus', 'banget', 'pokok']",1,bagus banget pokok
2,Game babi,negatif,permainan babi,"['main', 'babi']",-1,main babi
3,Keren,positif,keren,['keren'],1,keren
4,Tiktok. Pohon katus,netral,pohon katus,"['pohon', 'katus']",0,pohon katus
...,...,...,...,...,...,...
25911,Apk sesat mending download Coco fun :v,negatif,sesat lebih baik download coco fun v,"['sesat', 'lebih', 'baik', 'download', 'coco',...",-1,sesat lebih baik download coco fun v
25912,Seru2an dan bertambah teman virtual,positif,seruan bertambah teman virtual,"['seru', 'tambah', 'teman', 'virtual']",1,seru tambah teman virtual
25913,Ngentottt siateh,negatif,ngentot siateh,"['ngentot', 'siateh']",-1,ngentot siateh
25914,Gahbisa masuk akun aku,negatif,gahbisa masuk akun,"['gahbisa', 'masuk', 'akun']",-1,gahbisa masuk akun


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Kuliah/Proposal/TA/dataset/CleanData/df_tiktok_clean_used_3.csv')

In [ ]:
df

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
0,gp:AOqpTOH8YxnFy0h4a_g49dLt7raefmY3b3nhJEzZuqk...,Bagus bangettttttttttt pokonya,positif,2,bagus banget pokoknya,"['bagus', 'banget', 'pokok']",bagus banget pokok
1,gp:AOqpTOHzJ-osah_oYII8_mKFDvvLUNhL9FFo3dMYAJX...,Game babi,negatif,0,permainan babi,"['main', 'babi']",main babi
2,gp:AOqpTOEsUd_G8nZ3RdkMFdN9CEixth7_9FipWJpCgJ6...,Tiktok. Pohon katus,netral,1,pohon katus,"['pohon', 'katus']",pohon katus
3,gp:AOqpTOEWHfXgz1qYZfsXVHExYqXohT_XgbORTILu7NJ...,tolong jangan boros kali lah makan paket,negatif,0,boros kali makan paket,"['boros', 'kali', 'makan', 'paket']",boros kali makan paket
4,gp:AOqpTOHl33Ag2bFocaxz1d0VRR1TJ3wZlO0UwpBp77V...,Ngakak liat pendewa naruto kena infinite gojou...,netral,1,tertawa lihat pendewa naruto kena infinite goj...,"['tertawa', 'lihat', 'dewa', 'naruto', 'kena',...",tertawa lihat dewa naruto kena infinite gojou ...
...,...,...,...,...,...,...,...
22419,gp:AOqpTOHUr2AH7s0ANXk4YgiXAHVf4krAStLXXxbjcqr...,Apk sesat mending download Coco fun :v,negatif,0,sesat lebih baik download coco fun v,"['sesat', 'lebih', 'baik', 'download', 'coco',...",sesat lebih baik download coco fun v
22420,gp:AOqpTOHwMXaJSQ9tHpbwpaHzcq0aRzxci6xqh2DZ1n2...,Seru2an dan bertambah teman virtual,positif,2,seruan bertambah teman virtual,"['seru', 'tambah', 'teman', 'virtual']",seru tambah teman virtual
22421,gp:AOqpTOE3XpPZW4YJwJ5fqTYrvtX9Z-IAwuqi2tbw89D...,Ngentottt siateh,negatif,0,ngentot siateh,"['ngentot', 'siateh']",ngentot siateh
22422,gp:AOqpTOFa6V8C93IFNEyXY3iiFufXvvK_E-QnYLJbsQQ...,Gahbisa masuk akun aku,negatif,0,gahbisa masuk akun,"['gahbisa', 'masuk', 'akun']",gahbisa masuk akun


In [ ]:
df[['content','text_clean']].sample(5)

,content,text_clean
5094,"Aplikasi mntap,buat hiburan asek bnget",mntapbuat hiburan asyik banget
10527,"Tolong pak, apa cuman saya yanh yang kalo undu...",pak yanh unduh ulang gagal terus penyimpanan b...
3838,Kasih bintang tiga dulu nanti kalau sdh terbia...,bintang tiga terbiasa faham hadiah full mantap
6966,Jelek!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,jelektapi bohong wkwkwwk bagus
16656,"Appk keren, dan menghibur bgt",appk keren menghibur banget


In [ ]:
df[['content','text_used']].sample(5)

,content,text_used
11918,Iklan tidak di setujui tapi dana tetap berkurang,iklan tuju dana tetap kurang
16743,Semoga cepat fyp ya,moga cepat fyp
2558,tolong kasi fitur musik dan ngeliat siapa saja...,fitur musik lihat lihat akun doang beda
3171,Fitur tanggal lahir,fitur tanggal lahir
13912,Mohon konten yang mengandung 18+ akunnya di blok,kandung akun blok


#Cek Data Buat Jurnal

In [ ]:
df_net = df[df['label'] == 'positif']

In [ ]:
df_net.sample(10)

,reviewId,content,label,tag,text_clean,text_preprocessed,text_used
191,gp:AOqpTOEh5cXpBzVIGuENHkNag-iZaptkKn56D28tjAs...,"Tik Tok Ini Membuat Saya Terharu,Klo Mau Jadi ...",positif,2,membuat terharuklo jadi viral buruan donloat,"['buat', 'terharuklo', 'jadi', 'viral', 'buru'...",buat terharuklo jadi viral buru donloat
14311,gp:AOqpTOFn1VL4RW7JvM9eSNoJ37tqwnmkOkilwqazaA4...,Apk nya bagussssssssssssssssssss# Bangettttttt...,positif,2,bagus banget tt tt ttt bosenn terimakasih tij ...,"['bagus', 'banget', 'tt', 'tt', 'ttt', 'bosenn...",bagus banget tt tt ttt bosenn terimakasih tij ...
3325,gp:AOqpTOE1WWKQ4CK13V49uIu4bCQ0HX5WVoAkjfOBOi4...,Seru buat hiburan sama anak2 di rumah.,positif,2,seru hiburan anak rumah,"['seru', 'hibur', 'anak', 'rumah']",seru hibur anak rumah
12751,gp:AOqpTOHATk-pZ8TyM5Ab37QgyboBtuybL58iZXJGSyO...,Ok mantap tolong di saring lagi pengguna dibaw...,positif,2,ok mantap saring pengguna dibawa tahun,"['ok', 'mantap', 'saring', 'guna', 'bawa', 'ta...",ok mantap saring guna bawa tahun
7179,gp:AOqpTOFz3k7Ax6uLRMsmvAaGAFRc7Snz6sd-mEkE5Y8...,aku sngat suka ap ini ap sangat bagus😍😍😍😍😍😍😍😍😍...,positif,2,sngat suka bagus,"['sngat', 'suka', 'bagus']",sngat suka bagus
12037,gp:AOqpTOGaQ5Og9-w2M-Vd48BFTh53-cIo_CbY_Y5-1bB...,Terimakasih tiktok. Menjadi aplikasi andalan saya,positif,2,terimakasih menjadi andalan,"['terimakasih', 'jadi', 'andal']",terimakasih jadi andal
12362,gp:AOqpTOFdaE9LR7ZVbkyJ6OSpoLNM8SCQBFDOTUj6qcO...,"Saya suka sama apk ini, bisa membuat video² be...",positif,2,suka membuat video² berbagai cara mendapat bag...,"['suka', 'buat', 'video', 'bagai', 'cara', 'da...",suka buat video bagai cara dapat bagus kreatif
13780,gp:AOqpTOGSS5blcIt4LfrUK_e5IQ-Cv9FdHZsItFGVDbN...,Keren tapiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii...,positif,2,keren apaya,"['keren', 'apaya']",keren apaya
7986,gp:AOqpTOGQRHzfKAHLSmngwBPC0EQgH-EvZWQq5Abjap_...,Kalo mode gabut non tontikti D Tnrhdtn Mjt,positif,2,mode bosan non tontikti tnrhdtn mjt,"['mode', 'bosan', 'non', 'tontikti', 'tnrhdtn'...",mode bosan non tontikti tnrhdtn mjt
16769,gp:AOqpTOFq3l_ctUN5suwrsNjRrxN_6ys5YJqF-WIDtFK...,Bagus untuk meningkatkan mood sya Sya juga bis...,positif,2,bagus untuk meningkatkan mood jualan,"['bagus', 'untuk', 'tingkat', 'mood', 'jual']",bagus untuk tingkat mood jual


In [ ]:
df_net.loc[16769,'content']

'Bagus untuk meningkatkan mood sya Sya juga bisa jualan'